# 从0开始批量归一化

对于层数比较深的神经网络，我们通常使用批量归一化来增加数据稳定性。举个例子，随着第一层和第二层的参数在训练时不断变化，第三层所使用的激活函数的输入值可能由于乘法效应而变得极大或极小，例如和第一层所使用的激活函数的输入值不在一个数量级上。**这种在训练时可能出现的情况会造成模型训练的不稳定性。**例如，给定一个学习率，某次参数迭代后，目标函数值会剧烈变化或甚至升高。数学解释是，我们如果把目标函数$f$根据参数$\mathbf{w}$迭代$f(\mathbf{w} - \eta \nabla f(\mathbf{w}))$进行泰勒展开，有关学习率$\eta$的高阶项的系数可能由于数量级的原因（通常由于层数多）而不容忽略。然而常用的低阶优化算法（如梯度下降）对于不断降低目标函数的有效性通常基于一个基本假设：在以上泰勒展开中把有关学习率的高阶项通通忽略不计，这就直接导致了不同层间目标函数值的输出不在同一个量级上。

为了应对上述这种情况，Sergey Ioffe和Christian Szegedy在2015年提出了批量归一化的方法。简而言之，在训练时给定一个批量输入，批量归一化试图对深度学习模型的某一层所使用的激活函数的输入进行归一化：**<font color="red">使批量呈标准正态分布（均值为0，标准差为1）。**
    
批量归一化通常应用于输入层或任意中间层。

## 批量归一化层

根据原作者论文中的阐述，批量归一化层不同于丢弃层，通常在激活层之前使用。

其基本思想是对每一个批量($mini-batch$)的输入通过施加尺度变换$(scale\,and\,shift)$来进行归一化($normalization$)。

具体来说，对每一个输入批量$B = \{x_{1, ..., m}\}$，我们需要学习拉伸参数$\gamma$以及偏移参数$\beta$，输出层变为$\{y_i = BN_{\gamma, \beta}(x_i)\}$，其中：

$$\mu_B \leftarrow \frac{1}{m}\sum_{i = 1}^{m}x_i$$

$$\sigma_B^2 \leftarrow \frac{1}{m} \sum_{i=1}^{m}(x_i - \mu_B)^2$$

$$\hat{x_i} \leftarrow \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$$

$$y_i \leftarrow \gamma \hat{x_i} + \beta \equiv \mbox{BN}_{\gamma,\beta}(x_i)$$


请注意，对于全连接层，很明显我们需要对每个批量进行归一化。而对于2维卷积，我们需要对每个通道中的``batch_size * height * width``做归一化，因此$\gamma$和$\beta$的长度与通道数一致。在我们的实现中，我们需要``reshape``我们的$\gamma$和$\beta$，使得它们可以与矩阵正确的相乘，并且我们要保持输入的四维形状使得其可以正确地广播。

In [1]:
import mxnet as mx
import numpy as np

from mxnet import nd
from mxnet import gluon
from mxnet import autograd

mx.random.seed(1)
ctx = mx.gpu(0)

import utils

In [2]:
batch_size = 64
train_data, test_data = utils.load_dataset(batch_size, data_type='fashion_mnist')

In [3]:
def pure_batch_norm(X, gamma, beta, eps=1e-5):
    if len(X.shape) not in [2,4]:
        raise ValueError('the shape of the inputs must be 2 or 4.')
    
    if len(X.shape) == 2:
        mean = nd.mean(X, axis=0, keepdims=True)
        variance = nd.mean((X - mean)**2, axis=0, keepdims=True)
    else:
        mean = nd.mean(X, axis=(0,2,3), keepdims=True)
        variance = nd.mean((X - mean)**2, axis=(0,2,3), keepdims=True)
    
    X_hat = 1.0*(X - mean) / nd.sqrt(variance + eps)
    out = gamma.reshape(mean.shape) * X_hat + beta.reshape(mean.shape)
    return out

### 二维测试

In [4]:
X = nd.arange(4, ctx=ctx) + 1
X = X.reshape((2,2))
X


[[ 1.  2.]
 [ 3.  4.]]
<NDArray 2x2 @gpu(0)>

In [5]:
ga = nd.array([1,1], ctx=ctx)
be = nd.array([0,0], ctx=ctx)
pure_batch_norm(X, ga, be)


[[-0.99999499 -0.99999499]
 [ 0.99999499  0.99999499]]
<NDArray 2x2 @gpu(0)>

### 四维测试

In [6]:
B = nd.array([1,6,5,7,4,3,2,5,6,3,2,4,5,3,2,5,6], ctx=ctx).reshape((2,2,2,2))
B


[[[[ 1.  6.]
   [ 5.  7.]]

  [[ 4.  3.]
   [ 2.  5.]]]


 [[[ 6.  3.]
   [ 2.  4.]]

  [[ 5.  3.]
   [ 2.  5.]]]]
<NDArray 2x2x2x2 @gpu(0)>

In [7]:
pure_batch_norm(B, ga, be)


[[[[-1.63784397  0.88191599]
   [ 0.37796399  1.38586795]]

  [[ 0.30779248 -0.51298743]
   [-1.33376741  1.12857234]]]


 [[[ 0.88191599 -0.62993997]
   [-1.13389194 -0.12598799]]

  [[ 1.12857234 -0.51298743]
   [-1.33376741  1.12857234]]]]
<NDArray 2x2x2x2 @gpu(0)>

In [8]:
B2 = nd.arange(18, ctx=ctx).reshape((1,2,3,3))
B2


[[[[  0.   1.   2.]
   [  3.   4.   5.]
   [  6.   7.   8.]]

  [[  9.  10.  11.]
   [ 12.  13.  14.]
   [ 15.  16.  17.]]]]
<NDArray 1x2x3x3 @gpu(0)>

In [9]:
pure_batch_norm(B2, ga, be)


[[[[-1.54919219 -1.1618942  -0.7745961 ]
   [-0.38729805  0.          0.38729805]
   [ 0.7745961   1.1618942   1.54919219]]

  [[-1.54919219 -1.1618942  -0.7745961 ]
   [-0.38729805  0.          0.38729805]
   [ 0.7745961   1.1618942   1.54919219]]]]
<NDArray 1x2x3x3 @gpu(0)>

## 测试时的批量归一化

既然我们在训练时使用了批量归一化，那么一个很自然的问题是，我们在测试的时候需要使用批量归一化吗？

答案是肯定的，不用的话，训练出的模型在测试集上就不可能准确，但是在测试时，我们需要对批量归一化做些改动，一般在测试时，**我们使用的是整个训练集的均值和方差，而不再是每个批量的均值和方差**，但我们不会傻傻的计算整个训练集的均值和方差，因为这样会带来极大的计算开销(试想成百上千万的训练集)，因此我们使用移动平均的思想来计算，即在训练阶段我们除了计算每个批量的均值和方差意外，我们使用移动平均的思想来计算总的``moving_mean``和``moving_variance``，那么在测试阶段我们直接使用这两个结果来近似整个训练集的均值和方差。

下面我们实现批量归一化。

In [10]:
# def batch_norm(X, gamma, beta, moving_momentum=0.9, eps=1e-5, scope_name='', is_training=True):
#     # 定义全局变量；
#     global _BN_MOVING_MEANS, _BN_MOVING_VARS
    
#     # X的维度必须是2或4
#     assert len(X.shape) in [2,4]
    
#     if len(X.shape) == 2:
#         mean = nd.mean(X, axis=0)
#         variance = nd.mean((X - mean)**2, axis=0)
#     else:
#         mean = nd.mean(X, axis=(0,2,3), keepdims=True)
#         variance = nd.mean((X - mean)**2, axis=(0,2,3), keepdims=True)
         
#     #################### 全局变量 #########################    
#     try: # to access them
#         _BN_MOVING_MEANS, _BN_MOVING_VARS
#     except: # error, create them
#         _BN_MOVING_MEANS, _BN_MOVING_VARS = {}, {}    
        
#     if scope_name not in _BN_MOVING_MEANS:
#         _BN_MOVING_MEANS[scope_name] = mean
#     else:
#         _BN_MOVING_MEANS[scope_name] = _BN_MOVING_MEANS[scope_name] * moving_momentum + (1-moving_momentum) * mean
    
#     if scope_name not in _BN_MOVING_VARS:
#         _BN_MOVING_VARS[scope_name] = variance  
#     else:
#         _BN_MOVING_VARS[scope_name] = _BN_MOVING_VARS[scope_name] * moving_momentum + (1-moving_momentum) * variance
#     #################### 全局变量 #########################        
        
#     if is_training:
#         X_hat = (X - mean) / nd.sqrt(variance + eps)
#     else:
#         X_hat = (X - _BN_MOVING_MEANS[scope_name]) / nd.sqrt(_BN_MOVING_VARS[scope_name] + eps)
        
#     return gamma.reshape(mean.shape) * X_hat + beta.reshape(mean.shape)

In [11]:
def batch_norm(X, gamma, beta, is_training,  moving_mean, moving_variance,
               moving_momentum=0.9, eps=1e-5):
    if len(X.shape) not in [2,4]:
        raise ValueError("The dimension of the input must bt 2-d(Dense) or 4-d(Conv2d)")
        
    if len(X.shape) == 2:
        mean = nd.mean(X, axis=0, keepdims=True)
        variance = nd.mean((X - mean)**2, axis=0, keepdims=True)
    else:
        mean = nd.mean(X, axis=(0,2,3), keepdims=True)
        variance = nd.mean((X - mean)**2, axis=(0,2,3), keepdims=True)
    
    # 变形使得其可以正确广播             
    moving_mean = moving_mean.reshape(mean.shape)
    moving_variance = moving_variance.reshape(variance.shape)
    
    if is_training:
        X_hat = (X - mean) / nd.sqrt(variance + eps)
        # in-place operation
        moving_mean[:] = moving_momentum * moving_mean + (1-moving_momentum) * mean
        moving_variance[:] = moving_momentum * moving_variance + (1-moving_momentum) * variance
    else:
        X_hat = (X - moving_mean) / nd.sqrt(moving_variance + eps) 
        
    out = gamma.reshape(mean.shape) * X_hat + beta.reshape(mean.shape)
    return out

## 定义参数

* 权重缩减 : 0.01
* Conv1 : 20 output_channel, 3x3 kernel
* Conv2 : 50 output_channel, 5X5 kernel
* FC1 : 128 output
* FC2 : 10 output

### ``nd.random.normal``
* loc : 生成数据的概率分布的均值，对应着整个分布的中心。
* scale : 生成数据的概率分布的标准差，对应于分布的宽度，scale越大越矮胖，scale越小，越瘦高。

In [12]:
weight_scale =.01
num_output_conv1 = 20
num_output_conv2 = 50
num_output_fc1 = 128
num_output_fc2 = 10

############# first conv layer ############# 
W1 = nd.random.normal(shape=(num_output_conv1, 1, 3, 3), scale=weight_scale, ctx=ctx)
b1 = nd.random.normal(shape=num_output_conv1, scale=weight_scale, ctx=ctx)

gamma1 = nd.random.normal(shape=num_output_conv1, scale=weight_scale, ctx=ctx)
beta1 = nd.random.normal(shape=num_output_conv1, scale=weight_scale, ctx=ctx)

moving_mean1 = nd.zeros(num_output_conv1, ctx=ctx)
moving_variance1 = nd.zeros(num_output_conv1, ctx=ctx)

############# second conv layer ############# 
W2 = nd.random.normal(shape=(num_output_conv2, num_output_conv1, 5, 5), scale=weight_scale, ctx=ctx)
b2 = nd.random.normal(shape=num_output_conv2, scale=weight_scale, ctx=ctx)

gamma2 = nd.random.normal(shape=num_output_conv2, scale=weight_scale, ctx=ctx)
beta2 = nd.random.normal(shape=num_output_conv2, scale=weight_scale, ctx=ctx)

moving_mean2 = nd.zeros(num_output_conv2, ctx=ctx)  
moving_variance2 = nd.zeros(num_output_conv2, ctx=ctx)

############# first fc layer ############# 
W3 = nd.random.normal(shape=(800, num_output_fc1), scale=weight_scale, ctx=ctx)
b3 = nd.random.normal(shape=num_output_fc1, scale=weight_scale, ctx=ctx)

gamma3 = nd.random.normal(shape=num_output_fc1, scale=weight_scale, ctx=ctx)
beta3 = nd.random.normal(shape=num_output_fc1, scale=weight_scale, ctx=ctx)

# moving_mean3 = nd.zeros(num_output_fc1, ctx=ctx)
# moving_variance3 = nd.zeros(num_output_fc1, ctx=ctx)

############# second fc layer ############# 

W4 = nd.random.normal(shape=(num_output_fc1, num_output_fc2), scale=weight_scale, ctx=ctx)
b4 = nd.random.normal(shape=num_output_fc2, scale=weight_scale, ctx=ctx)

# moving_loss 和\ moving_variance不需要更新
params = [W1, b1, gamma1, beta1, W2, b2, gamma2, beta2, W3, b3, W4, b4]

for param in params:
    param.attach_grad()

In [13]:
# shape inference
X = nd.random.normal(shape=(32,1,28,28), ctx=ctx)
conv1 = nd.Convolution(data=X, weight=W1, bias=b1, kernel=W1.shape[2:], stride=(1,1), 
                       num_filter=W1.shape[0])
pool1 = nd.Pooling(data=conv1, kernel=(2,2), pool_type='max', stride=(2,2))
conv2 = nd.Convolution(data=pool1, weight=W2, bias=b2, kernel=W2.shape[2:], stride=(1,1), 
                       num_filter=W2.shape[0])
pool2 = nd.Pooling(data=conv2, kernel=(2,2), pool_type='max', stride=(2,2))
print(pool2.shape)

(32, 50, 4, 4)


## 定义模型

conv -> norm -> relu -> pool

In [14]:
def relu(X):
    return nd.maximum(X, nd.zeros_like(X))

In [15]:
def net(X, is_training=False, verbose=False):
    ############# first conv layer ############# 
    conv1 = nd.Convolution(data=X, weight=W1, bias=b1, kernel=W1.shape[2:], stride=(1,1), 
                       num_filter=W1.shape[0])
    norm1 = batch_norm(conv1, gamma1, beta1, is_training, moving_mean1, moving_variance1)
    relu1 = relu(norm1)
    pool1 = nd.Pooling(data=relu1, pool_type='avg', kernel=(2,2), stride=(2,2))
    
    
    ############# second conv layer ############# 
    conv2 = nd.Convolution(data=pool1, weight=W2, bias=b2, kernel=W2.shape[2:], stride=(1,1), 
                       num_filter=W2.shape[0])
    norm2 = batch_norm(conv2, gamma2, beta2, is_training, moving_mean2, moving_variance2)
    relu2 = relu(norm2)
    pool2 = nd.Pooling(data=relu2, pool_type='avg', kernel=(2,2), stride=(2,2))

    
    ############# flatten layer #############
    flatten = nd.Flatten(pool2)
    
    ############# first fc layer #############
    fc1 = nd.dot(flatten, W3) + b3
    # norm3 = batch_norm(fc1, gamma3, beta3, is_training, moving_mean3, moving_variance3)
    relu3 = relu(fc1)
    
    ############# second fc layer #############
    fc2 = nd.dot(relu3, W4) + b4
    
    if verbose:
        print("input shape : ", X.shape)
        print("conv1 shape : ", conv1.shape)
        print("pool1 shape : ", pool1.shape)
        print("conv2 shape : ", conv2.shape)
        print("pool2 shape : ", pool2.shape)
        print("fc1 shape : ", fc1.shape)
        print("output shape : ", fc2.shape)
        
    return fc2

## 定义优化器和损失函数

In [16]:
def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

In [ ]:
def softmax(ylinear):
    yexp = nd.exp(ylinear - nd.max(ylinear))
    partition = nd.nansum(yexp, axis=1, keepdims=True) # keepdims相当于reshape((-1, 1))
    return yexp / partition  

def softmax_cross_entropy(yhat, y): 
    return - nd.nansum(y * nd.log(softmax(yhat)), axis=1)

## 训练

In [ ]:
from time import time

epochs = 10
learning_rate = 0.1

niter = 0
moving_loss = .0
smoothing_constant = 0.9   

for epoch in range(epochs):
    start = time()
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        label_one_hot = nd.one_hot(label, 10)
        with autograd.record():
            # 训练阶段
            output = net(data, is_training=True)
            loss = softmax_cross_entropy(output, label_one_hot)
        loss.backward()
        SGD(params, learning_rate)
        
        niter += 1
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = smoothing_constant * moving_loss + (1 - smoothing_constant) * curr_loss
        estimated_loss = moving_loss / (1 - smoothing_constant**niter)
      
    train_acc = utils.evaluate_accuracy_scratch(train_data, net, ctx)
    test_acc = utils.evaluate_accuracy_scratch(test_data, net, ctx)
    print("Epoch %d, Moving Train Avg loss %.5f, Train acc %.5f, Test acc %.5f, Time consume %.5f s."
         % (epoch, estimated_loss, train_acc, test_acc, time() - start))

## 2018年3月23日更新

Kaiming He等人提出组归一化Group Norm

论文地址：https://arxiv.org/abs/1803.08494

In [5]:
def group_norm(X, gamma, beta, G, eps=2e-5):
    assert len(X.shape) == 4
    N, C, H, W = X.shape
    
    X = X.reshape((N, G, C // G, H, W))
    mean = nd.mean(X, axis=(2, 3, 4), keepdims=True)
    variance = nd.mean((X - mean)**2, axis=(2, 3, 4), keepdims=True)
    
    X_hat = 1.0*(X - mean) / nd.sqrt(variance + eps)
    out = gamma.reshape(mean.shape) * X_hat + beta.reshape(mean.shape)
    out = out.reshape((N, C, H, W))
    return out